<a href="https://colab.research.google.com/github/dk3059/Machine-Learning/blob/main/1_1_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%B6%84%EB%A5%98%EB%AA%A8%ED%98%95_US_Election_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 데이터 준비

### 데이터 사용 파일
 - president_country_candidate.csv : 대통령 투표 결과
 - governors_country_candidate.csv : 카운티 자사 투표 결과
 - senate_country_candidate.csv : 상원의원 투표 결과
 - house_candidate.csv : 하원의원 투표 결과




 ### 컬럼 설명
  - state : 주
  - country : 카운티(군)
  - district : 지구
  - candidate : 후보자
  - party : 후보자의 소속 정당
  - total_votes : 득표 수
  - won : 지역 투표 우승 여부

In [2]:
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install plotly-geo

     |████████████████████████████████| 1.0MB 4.3MB/s 
     |████████████████████████████████| 15.3MB 247kB/s 
     |████████████████████████████████| 6.5MB 39.0MB/s 
     |████████████████████████████████| 225kB 4.3MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37264 sha256=bbeee9861815616b133e2a005bf03421dafcd30eb0279750ea0ef2c85edc1689
  Stored in directory: /root/.cache/pip/wheels/76/2b/d4/53e6b9a0fb0a9f9f29664cf82605af8bc81d5ab44d987896dd
Successfully built pyshp
     |████████████████████████████████| 23.7MB 60.9MB/s 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import os

In [7]:
!kaggle datasets download -d unanimad/us-election-2020
!kaggle datasets download -d muonneutrino/us-census-demographic-data

  0% 0.00/429k [00:00<?, ?B/s]
100% 429k/429k [00:00<00:00, 54.1MB/s]
 85% 9.00M/10.6M [00:00<00:00, 12.7MB/s]
100% 10.6M/10.6M [00:00<00:00, 16.6MB/s]


In [8]:
!unzip '*.zip'

Archive:  us-election-2020.zip
  inflating: governors_county.csv    
  inflating: governors_county_candidate.csv  
  inflating: governors_state.csv     
  inflating: house_candidate.csv     
  inflating: house_state.csv         
  inflating: president_county.csv    
  inflating: president_county_candidate.csv  
  inflating: president_state.csv     
  inflating: senate_county.csv       
  inflating: senate_county_candidate.csv  
  inflating: senate_state.csv        

Archive:  us-census-demographic-data.zip
  inflating: acs2015_census_tract_data.csv  
  inflating: acs2015_county_data.csv  
  inflating: acs2017_census_tract_data.csv  
  inflating: acs2017_county_data.csv  

2 archives were successfully processed.


In [51]:
# US Election 2020
df_pres = pd.read_csv('president_county_candidate.csv')
df_gov = pd.read_csv('governors_county_candidate.csv')

#US Census 2017
df_census = pd.read_csv('acs2017_county_data.csv')

In [13]:
#State Code 관련 부가 자료
state_code = pd.read_html('https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes')[0]

##1. EDA

In [29]:
df_pres.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [30]:
df_pres['candidate'].unique()

array(['Joe Biden', 'Donald Trump', 'Jo Jorgensen', 'Howie Hawkins',
       ' Write-ins', 'Gloria La Riva', 'Brock Pierce',
       'Rocky De La Fuente', 'Don Blankenship', 'Kanye West',
       'Brian Carroll', 'Ricki Sue King', 'Jade Simmons',
       'President Boddie', 'Bill Hammons', 'Tom Hoefling',
       'Alyson Kennedy', 'Jerome Segal', 'Phil Collins',
       ' None of these candidates', 'Sheila Samm Tittle', 'Dario Hunter',
       'Joe McHugh', 'Christopher LaFontaine', 'Keith McCormic',
       'Brooke Paige', 'Gary Swing', 'Richard Duncan', 'Blake Huber',
       'Kyle Kopitke', 'Zachary Scalf', 'Jesse Ventura', 'Connie Gammon',
       'John Richard Myers', 'Mark Charles', 'Princess Jacob-Fambro',
       'Joseph Kishore', 'Jordan Scott'], dtype=object)

In [31]:
df_gov.head()

,state,county,candidate,party,votes,won
0,Delaware,Kent County,John Carney,DEM,44352,True
1,Delaware,Kent County,Julianne Murray,REP,39332,False
2,Delaware,Kent County,Kathy DeMatteis,IPD,1115,False
3,Delaware,Kent County,John Machurek,LIB,616,False
4,Delaware,New Castle County,John Carney,DEM,191678,True


In [32]:
df_census.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [33]:
df_census['County'].value_counts()

Washington County    30
Jefferson County     25
Franklin County      24
Jackson County       23
Lincoln County       23
                     ..
Colleton County       1
Stanly County         1
La Crosse County      1
Marathon County       1
Reagan County         1
Name: County, Length: 1955, dtype: int64

In [34]:
state_code.head()

,State/District,Abbreviation,Postal Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [57]:
# 모든 데이터프레임의 index를 country로 변경하기

data = df_pres.loc[df_pres['party'].apply(lambda s : str(s) in ['DEM', 'REP'])] #민주당, 공화당만 남기기
table_pres = pd.pivot_table(data=data, index = ['state', 'county'], columns = 'party', values='total_votes')
table_pres.rename({'DEM': 'PresDEM', 'REP':'Pres_REP'}, axis=1, inplace=True)
table_pres.isna().sum()

party
PresDEM     0
Pres_REP    0
dtype: int64

In [42]:
table_pres.head()

party                     DEM    REP
state   county                      
Alabama Autauga County   7503  19838
        Baldwin County  24578  83544
        Barbour County   4816   5622
        Bibb County      1986   7525
        Blount County    2640  24711

In [56]:
data = df_gov.loc[df_gov['party'].apply(lambda s : str(s) in ['DEM', 'REP'])]
table_gov = pd.pivot_table(data=data, index=['state', 'county'], columns='party', values='votes') #지역별 투표수
table_gov.rename({'DEM': 'PresDEM', 'REP':'Pres_REP'}, axis=1, inplace=True)
table_gov

party                            PresDEM  Pres_REP
state         county                              
Delaware      Kent County          44352     39332
              New Castle County   191678     82545
              Sussex County        56873     68435
Indiana       Adams County          2143      9441
              Allen County         53895     98406
...                                  ...       ...
West Virginia Webster County         659      2552
              Wetzel County         1727      4559
              Wirt County            483      1947
              Wood County           9933     26232
              Wyoming County        1240      6941

[1025 rows x 2 columns]

In [52]:
df_census.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [53]:
df_census.rename({'State' : 'state', 'County' : 'county'}, axis=1, inplace=True)
df_census.drop('CountyId', axis=1, inplace=True)
df_census.set_index(['state', 'county'], inplace=True)
df_census.head()

TotalPop    Men  ...  FamilyWork  Unemployment
state   county                           ...                          
Alabama Autauga County     55036  26899  ...         0.1           5.2
        Baldwin County    203360  99527  ...         0.1           5.5
        Barbour County     26201  13976  ...         0.3          12.4
        Bibb County        22580  12251  ...         0.3           8.2
        Blount County      57667  28490  ...         0.1           4.9

[5 rows x 34 columns]

In [58]:
df_census.columns

Index(['TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native',
       'Asian', 'Pacific', 'VotingAgeCitizen', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment'],
      dtype='object')

In [59]:
df_census.drop(['Income', 'IncomeErr', 'IncomePerCapErr'], axis=1, inplace=True)

In [60]:
# 다중공성선 피하기
df_census.drop('Women', axis=1, inplace=True)
df_census['Men'] /= df_census['TotalPop']
df_census['VotingAgeCitizen'] /= df_census['TotalPop']
df_census['Employed'] /= df_census['TotalPop']

In [62]:
df_census.head()

TotalPop       Men  ...  FamilyWork  Unemployment
state   county                              ...                          
Alabama Autauga County     55036  0.488753  ...         0.1           5.2
        Baldwin County    203360  0.489413  ...         0.1           5.5
        Barbour County     26201  0.533415  ...         0.3          12.4
        Bibb County        22580  0.542560  ...         0.3           8.2
        Blount County      57667  0.494043  ...         0.1           4.9

[5 rows x 30 columns]

In [63]:
# df_pres, df_gov, df_census 데이터프레임 합치기
df = pd.concat([table_pres, table_gov, df_census], axis=1)

In [64]:
df.head()

PresDEM  Pres_REP  ...  FamilyWork  Unemployment
state   county                             ...                          
Alabama Autauga County   7503.0   19838.0  ...         0.1           5.2
        Baldwin County  24578.0   83544.0  ...         0.1           5.5
        Barbour County   4816.0    5622.0  ...         0.3          12.4
        Bibb County      1986.0    7525.0  ...         0.3           8.2
        Blount County    2640.0   24711.0  ...         0.1           4.9

[5 rows x 34 columns]